# site — Site-wide Configuration

## Import Path

site is automatically imported each time the interpreter starts up. On import, it extends sys.path with site-specific names constructed by combining the prefix values sys.prefix and sys.exec_prefix with several suffixes. The prefix values used are saved in the module-level variable PREFIXES for reference later. 

Under Windows, the suffixes are an empty string and lib/site-packages. 
For Unix-like platforms, the values are lib/python$version/site-packages (where $version is replaced by the major and minor version number of the interpreter, such as 3.5) and lib/site-python.

In [7]:
# site_import_path.py
import sys
import os
import site

if 'Windows' in sys.platform:
    SUFFIXES = [
        '',
        'lib/site-packages',
    ]
else:
    SUFFIXES = [
        'lib/python{}/site-packages'.format(sys.version[:3]),
        'lib/site-python',
    ]

print('Path prefixes:')
for p in site.PREFIXES:
    print('  ', p)

for prefix in sorted(set(site.PREFIXES)):
    print()
    print('prefix:', prefix)
    for suffix in SUFFIXES:
        print()
        print('suffix:', suffix)
        path = os.path.join(prefix, suffix).rstrip(os.sep)
        print('Full path:', path)
        print('   exists :', os.path.exists(path))
        print('   in path:', path in sys.path)

Path prefixes:
   /Users/binyang/anaconda/envs/py3.5
   /Users/binyang/anaconda/envs/py3.5

prefix: /Users/binyang/anaconda/envs/py3.5

suffix: lib/python3.5/site-packages
Full path: /Users/binyang/anaconda/envs/py3.5/lib/python3.5/site-packages
   exists : True
   in path: True

suffix: lib/site-python
Full path: /Users/binyang/anaconda/envs/py3.5/lib/site-python
   exists : False
   in path: False


## User Directories

In addition to the global site-packages paths, site is responsible for adding the user-specific locations to the import path. The user-specific paths are all based on the USER_BASE directory, which usually located in a part of the file system owned (and writable) by the current user. Inside the USER_BASE directory is a site-packages directory, with the path accessible as USER_SITE.

In [8]:
# site_user_base.py
import site

print('Base:', site.USER_BASE)
print('Site:', site.USER_SITE)

Base: /Users/binyang/.local
Site: /Users/binyang/.local/lib/python3.5/site-packages


In [11]:
# site_enable_user_site.py
import site

status = {
    None: 'Disabled for security',
    True: 'Enabled',
    False: 'Disabled by command-line option',
}

print('Flag   :', site.ENABLE_USER_SITE)
print('Meaning:', status[site.ENABLE_USER_SITE])

Flag   : True
Meaning: Enabled


## Path Configuration Files

As paths are added to the import path, they are also scanned for path configuration files. A path configuration file is a plain text file with the extension .pth. Each line in the file can take one of four forms.

* A full or relative path to another location that should be added to the import path.
* A Python statement to be executed. All such lines must begin with an import statement.
* Blank lines are ignored.
* A line starting with # is treated as a comment and ignored.

Path configuration files can be used to extend the import path to look in locations that would not have been added automatically. For example, the setuptools package adds a path to easy-install.pth when it installs a package in development mode using python setup.py develop.

## Customizing Site Configuration

The site module is also responsible for loading site-wide customization defined by the local site owner in a sitecustomize module. Uses for sitecustomize include extending the import path and enabling coverage, profiling, or other development tools.

In [26]:
# with_sitecustomize/sitecustomize.py
print('Loading sitecustomize.py')

import site
import platform
import os
import sys

path = os.path.join('/opt',
                    'python',
                    sys.version[:3],
                    platform.platform(),
                    )
print('Adding new path', path)

site.addsitedir(path)

Loading sitecustomize.py
Adding new path /opt/python/3.5/Darwin-16.3.0-x86_64-i386-64bit


In [29]:
# with_sitecustomize/site_sitecustomize.py
import sys

print('Running main program from\n{}'.format(sys.argv[0]))

print('End of path:', sys.path[-1])

Running main program from
/Users/binyang/anaconda/envs/py3.5/lib/python3.5/site-packages/ipykernel/__main__.py
End of path: /opt/python/3.5/Darwin-16.3.0-x86_64-i386-64bit


## Customizing User Configuration¶

Similar to sitecustomize, the usercustomize module can be used to set up user-specific settings each time the interpreter starts up. usercustomize is loaded after sitecustomize, so site-wide customizations can be overridden.

In environments where a user’s home directory is shared on several servers running different operating systems or versions, the standard user directory mechanism may not work for user-specific installations of packages. In these cases, a platform-specific directory tree can be used instead.

In [30]:
# with_usercustomize/usercustomize.py
print('Loading usercustomize.py')

import site
import platform
import os
import sys

path = os.path.expanduser(os.path.join('~',
                                       'python',
                                       sys.version[:3],
                                       platform.platform(),
                                       ))
print('Adding new path', path)

site.addsitedir(path)


Loading usercustomize.py
Adding new path /Users/binyang/python/3.5/Darwin-16.3.0-x86_64-i386-64bit


In [32]:
sys.path

['',
 '/Users/binyang/anaconda/envs/py3.5/lib/python35.zip',
 '/Users/binyang/anaconda/envs/py3.5/lib/python3.5',
 '/Users/binyang/anaconda/envs/py3.5/lib/python3.5/plat-darwin',
 '/Users/binyang/anaconda/envs/py3.5/lib/python3.5/lib-dynload',
 '/Users/binyang/anaconda/envs/py3.5/lib/python3.5/site-packages',
 '/Users/binyang/anaconda/envs/py3.5/lib/python3.5/site-packages/Sphinx-1.5.1-py3.5.egg',
 '/Users/binyang/anaconda/envs/py3.5/lib/python3.5/site-packages/setuptools-27.2.0-py3.5.egg',
 '/Users/binyang/anaconda/envs/py3.5/lib/python3.5/site-packages/IPython/extensions',
 '/Users/binyang/.ipython',
 '/opt/python/3.5/Darwin-16.3.0-x86_64-i386-64bit',
 '/Users/binyang/python/3.5/Darwin-16.3.0-x86_64-i386-64bit']

## Disabling the site Module

To maintain backwards-compatibility with versions of Python from before the automatic import was added, the interpreter accepts an -S option.

$ python3 -S site_import_path.py